## process pickle result for statistics info

In [4]:
import os
import pickle
import re
# load pickle
class my_pickle:
    def __init__(self, file_path) -> None:
        with open(file_path, 'rb') as f:
            result = pickle.load(f)
        if isinstance(result, list):
            version = "old"
        elif isinstance(result, dict):
            version = "new"
        else:
            raise ValueError("not dict or list type")
        if version == "old":
            [self.correct_num, self.error_num, self.answers_list, self.contents_list] = result
        else:
            self.correct_num = result["correct_num"]
            self.error_num = result["error_num"]
            self.answers_list = result["answers_list"]
            self.contents_list = result["contents_list"]
        file_name = os.path.basename(file_path)
        
        # match = re.match(r'case(\d+)(?:_temp\d_n(\d+))?', file_name)
        # if match:
        # # 获取与'case'后面的数字匹配的部分
        #     case_num = match.group(1)
        #     # 获取与'n'后面的数字匹配的部分，如果没有匹配到则默认为'5'
        #     n_number = match.group(2) if match.group(2) else '5'
        #     print(f"Matched case number: {case_num}, n number: {n_number}")
        # self.case_num = case_num
        # self.n_reason = n_number
    
        
    def avg_acc(self, case_num, reason_num):
        correct_num = 0
        error_num = 0
        for case_id in range(case_num):
            case_answers = self.answers_list[case_id]
            for answer_id in range(reason_num):
                answer = case_answers[answer_id]
                if isinstance(answer, list):
                    if answer[1] == 1:
                        correct_num += 1  
                    else: 
                        error_num += 1
        print("acc:{}, correct num: {}, err num: {}".format(correct_num / (case_num * reason_num), correct_num, error_num))
        
        return correct_num / (case_num * reason_num), correct_num, error_num
    
    def vote(self, case_num, reason_num):
        from collections import Counter
        valid_count = 0
        vote_right_count = 0
        for case_id in range(case_num):
            answers = self.answers_list[case_id][:reason_num]
            answers = [answer for answer in answers if isinstance(answer, list)]  # clean answers
            if len(answers) == 0:
                continue
            else:
                valid_count += 1
            second_elements_are_1 = [sublist[1] == 1 for sublist in answers]
            any_second_element_is_1 = any(second_elements_are_1)
            if not any_second_element_is_1:
                continue
            true_answer = next((sublist[0] for sublist in answers if sublist[1] == 1), None)
            counter = Counter(sublist[0] for sublist in answers)
            guess_value, _ = counter.most_common(1)[0]
            if guess_value == true_answer:
                vote_right_count += 1

        print("vote_correct_num:{}, vaild_num:{}, vote_correct_num:{}".format(vote_right_count, valid_count, vote_right_count / valid_count))        
        

In [76]:
def reset_acc(file_path, case_num, reason_num):
    pattern = re.compile(r'index\s\d+/\d+,\scorrect_num\s\d+,\serror_num\s\d+')
    pattern2 = re.compile(r', correate_rate:0.\d+')
    accuracy_pattern = re.compile(r'accuracy[^,]*,?')
    # correate_rate_pattern = re.compile(r'correate_rate:[^,]*,?')
    with open(file_path, 'r', encoding="utf-8") as file:
        lines = file.readlines()
    new_lines = []
    for line in lines:
        if pattern.search(line):
            # 如果行包含'accuracy'
            if 'accuracy' in line:
                line = accuracy_pattern.sub('', line)
            index = int(re.search(r'index\s(\d+)/\d+', line).group(1))
            correct_num = int(re.search(r'correct_num\s(\d+)', line).group(1))
            error_num = int(re.search(r'error_num\s(\d+)', line).group(1))
            if "Acc" not in line:
                Acc = correct_num/((index+1) * 5 * reason_num)
                line = line.rstrip('\n') + f', Acc {correct_num/((index+1) * 5 * reason_num)}\n'
        if pattern2.search(line):
            print("123")
            print(line)
            line = pattern2.sub('', line)
            line = line.rstrip('\n') + f', Acc: {Acc}\n'
        new_lines.append(line)
    file_name = os.path.basename(file_path)
    file_dir = os.path.dirname(file_path)
    if not os.path.exists(os.path.join(file_dir, "new")):
        os.makedirs(os.path.join(file_dir, "new"))
    with open(os.path.join(file_dir, "new", file_name), 'w') as file:
        file.writelines(new_lines)

In [125]:
dataset = "family_relation"
sub_task = "symbolic/hop3"
method = "selfdenoise"
case_num = 300
if_ICL = True
shot_num = 3
if_noise = False
noise_type = "inaccurate"
noise_ratio = 0.5
reason_num = 10

if not if_ICL:
    file_path = f"./result/{dataset}/{sub_task}/gpt-3.5-turbo-0613/method_{method}/log_origin_case{case_num}"
else:    
    if not if_noise:
        file_path = f"./result/{dataset}/{sub_task}/gpt-3.5-turbo-0613/method_{method}/log_ICL_{shot_num}clean_origin_case{case_num}"
    else:
        file_path = f"./result/{dataset}/{sub_task}/gpt-3.5-turbo-0613/method_{method}/log_ICL_0clean_noise_3{noise_type}_fixed_ratio{noise_ratio}_case{case_num}"
if method == "basemodel":
    file_path += f"_temp1_n{reason_num}"
log_file = file_path+".log"
file_path += ".pkl"

print(file_path)
pkl = my_pickle(file_path)
print(pkl.correct_num, pkl.error_num)
reset_acc(log_file, case_num, reason_num)



./result/family_relation/symbolic/hop3/gpt-3.5-turbo-0613/method_selfdenoise/log_ICL_3clean_origin_case300.pkl
1334 1619
123
correct_num:1334, error_num:1619, correate_rate:0.45174398916356245



: 

In [42]:
# save_case_num = 300
# save_reason_num = 5
pkl.avg_acc(300, 10)
# file_path =  f"./result/base_math/base-9/gpt-3.5-turbo-0613/method_basemodel/log_ICL_0clean_noise_3{noise_type}_fixed_ratio{noise_ratio}_case{save_case_num}_temp1_n{save_reason_num}.log"

acc:0.059666666666666666, correct num: 179, err num: 2802


(0.059666666666666666, 179, 2802)

In [43]:
# for vote
pkl.vote(300, 5)

vote_correct_num:26, vaild_num:300, vote_correct_num:0.08666666666666667
